In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
a = 5
b = 2
c = 5

In [ ]:
tf.random.set_seed(90)
M = tf.Variable(tf.random.normal((a, b, c), 5, 2), name='M')
p = tf.Variable(tf.random.uniform((), 0, 1, name='p'))

In [ ]:
with tf.device("/cpu:0"):
    with tf.GradientTape(persistent=True) as tape:
        reshaped_M = tf.reshape(M, [a*b, -1])  # dropping one dimension to ease the operations on the last dimension
        N = tf.reshape(tf.map_fn(lambda row: tf.scan(lambda x, y: x*p+y, row), reshaped_M), M.shape)
print('dN/dM:')
print(tape.gradient(N, M))
print('dN/dp:', tape.gradient(N, p).numpy())

In [ ]:
N

## The same, but implemented with numpy - for verification purposes

In [ ]:
memory_accumulation = np.frompyfunc(lambda x, y: y + p.numpy()*x, 2, 1)

In [ ]:
N_numpy = memory_accumulation.accumulate(M.numpy(), axis=2, dtype=object).astype(float)

In [ ]:
np.allclose(N.numpy(), N_numpy)  # exact comparison doesn't work for numerical reasons